In [1]:
import numpy as np
import netket as nk
import functions as f

/home/keisuke/Documents/Research/sshfs/DimerMaster/functions.py:65: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [2]:
from conf import *
from numba import njit

In [3]:
np.zeros(3, dtype=np.intp)[0:3]
np.random.rand(4)
5.1 //3

1.0

In [4]:
@njit
def foo(x,y):
    print((x//y).astype(np.int64))
    return x

In [5]:
x = np.random.randint(0,100, 10)
y = np.random.randint(1,10, 10)

In [6]:
foo(x,y)

[15 14 17  2  2 13 83 10  4  1]


array([92, 71, 71, 13,  8, 95, 83, 92, 21,  1])

In [7]:
h = 1.0
V = 1.0

length = [4, 4]
name = 'h={}V={}l={}'.format(h, V, length)
n_chains = 10
sweep_size = 70
n_samples = 10000



g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))


ma = nk.machine.RbmDimer(hi, hex_, alpha = alpha, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float)

ma.load('save/ma/'+name)


sa = nk.sampler.DimerMetropolisLocal(machine=ma, op=op_transition, length = length, n_chains=n_chains, sweep_size = 70)
sa.generate_samples(10) # discard the begginings of metropolis sampling.
print('discard samples')
samples_state = sa.generate_samples(int(n_samples / n_chains))
samples_state = samples_state.reshape(-1, ma.hilbert.size)

print('prepared initial samples')




4
discard samples
prepared initial samples


In [8]:
import numpy as _np
@njit
def _get_conn_flattened_kernel(
        x,
        sections,
        local_states,
        basis,
        constant,
        diag_mels,
        n_conns,
        all_mels,
        all_x_prime,
        acting_on
    ):
    
    
    batch_size = x.shape[0]
    n_sites = x.shape[1]

    assert sections.shape[0] == batch_size

    n_operators = n_conns.shape[0]
    xs_n = _np.empty((batch_size, n_operators), dtype=_np.intp)

    max_conn = 0

    acting_size = 4
    tot_conn = 0

    sections += 1
    for i in range(n_operators):
        n_conns_i = n_conns[i]
        x_i = (x[:, acting_on[i]] + 1) / 2
        s = np.zeros(batch_size)
        for j in range(acting_size):
            s += x_i[:, j] * basis[j]
        xs_n[:, i] = s
        sections += n_conns_i[xs_n[:, i]]
    tot_conn=sections.sum()
    
    s = 0 
    for i in range(batch_size):
        s += sections[i]
        sections[i] = s

    
    
#     for b in range(batch_size):
#         # diagonal element
#         conn_b = 1

#         # counting the off-diagonal elements
#         for i in range(n_operators):
#             x_b = x[b]
#             x_i = x_b[acting_on[i]]
            
            
# #             xs_n[b, i] = (_np.searchsorted(local_states, x_i) * basis).sum()

#             s = 0
#             for k in range(acting_size):
#                  s += (
#                     (x_i[k] + 1)/2
#                     * basis[k]
#                 )
#                 # print('xs_n',xs_n[b, i], ' x_i', x_i[acting_size_i - k - 1])
#             xs_n[b, i] = s
#             conn_b += n_conns[i, xs_n[b, i]]

#         tot_conn += conn_b
#         sections[b] = tot_conn



    x_prime = _np.empty((tot_conn, n_sites), dtype=_np.int8) # x.shpae[0] is number of connected elements of hamiltonian from batch of states. 
    mels = _np.empty(tot_conn, dtype=_np.complex128)

    c = 0
    for b in range(batch_size):
        c_diag = c
        mels[c_diag] = constant
        x_batch = x[b]
        x_prime[c_diag] = _np.copy(x_batch)
        c += 1
        for i in range(n_operators):

            # Diagonal part
            mels[c_diag] += diag_mels[i, xs_n[b, i]]
            n_conn_i = n_conns[i, xs_n[b, i]]

            if n_conn_i > 0:
                sites = acting_on[i]

                for cc in range(n_conn_i):
                    mels[c + cc] = all_mels[i, xs_n[b, i], cc]
                    x_prime[c + cc] = _np.copy(x_batch) # assigin original basis to n_conn_i number of vectors, where n_conn_i means number of connected basis from one local operator indexed by i. 

                    for k in range(acting_size): 
                        x_prime[c + cc, sites[k]] = all_x_prime[
                            i, xs_n[b, i], cc, k
                        ]
                c += n_conn_i


    return x_prime, mels, sections

In [17]:
%timeit np.searchsorted(np.array([-1, 1]), samples_state)

1.18 ms ± 3.03 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [18]:
%timeit (samples_state + 1 ) / 2

226 µs ± 1.82 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
samples_state.shape

(10000, 32)

In [9]:
samples_state[:,op._acting_on[0]]

array([[-1., -1.,  1., -1.],
       [-1., -1., -1.,  1.],
       [-1.,  1.,  1., -1.],
       ...,
       [-1., -1.,  1., -1.],
       [ 1.,  1.,  1., -1.],
       [ 1., -1., -1.,  1.]])

In [10]:
batch_size = samples_state.shape[0]
basis = op._basis[::-1].copy()

In [13]:
%%timeit
sections = np.zeros(batch_size, dtype=np.int64 )
samples_state = np.random.choice([-1,1], (10000,32)).astype(np.int64)
x_prime, mels, sec = _get_conn_flattened_kernel(
    samples_state,
    sections,
    op._local_states,
    basis,
    op._constant,
    op._diag_mels,
    op._n_conns,
    op._mels,
    op._x_prime,
    op._acting_on)

30.5 ms ± 315 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
sections = np.zeros(batch_size, dtype=np.int64 )
samples_state = np.random.choice([-1,1], (10000,32)).astype(np.int64)

1.11 ms ± 3.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
np.random.choice([-1,1], (10000,4))

array([[-1,  1, -1,  1],
       [-1, -1, -1, -1],
       [-1,  1, -1,  1],
       ...,
       [ 1, -1, -1, -1],
       [-1, -1, -1,  1],
       [-1,  1,  1,  1]])

In [17]:
%%timeit 
sections = np.zeros(batch_size, dtype=np.int64 )
samples_state = np.random.choice([-1,1], (10000,32)).astype(np.int64)
x_prime, mels = op.get_conn_flattened(samples_state, sections)

70.4 ms ± 160 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
(x_prime_ == x_prime).all()

NameError: name 'x_prime_' is not defined

In [ ]:
sections

In [ ]:
sections